In [1]:
import numpy as np
import pandas as pd
import glob, os

In [2]:
'''
Calculate individual values for each larva's entire trajectory and save to a master CSV file. 
'''

# read in the animal_IDs of all individual larvae
df = pd.read_csv('./static_data_all_GCAMP.csv')

# remove any larvae that haven't been analyzed yet in Multitracker,
# for either the acclimate or experiment trials. 
df['fname1'] = './gcamp_video_calculations/' + df['animal_ID'] + '-acclimate.csv'
df['fname2'] = './gcamp_video_calculations/' + df['animal_ID'] + '-experiment.csv'
df['isfile1'] = df['fname1'].apply(os.path.isfile)
df['isfile2'] = df['fname2'].apply(os.path.isfile)

# Print those with missing files for acclimation
print('Missing acclimation files')
print(df[df['isfile1'] == False]['animal_ID'])
print('Missing experiment files')
print(df[df['isfile2'] == False]['animal_ID'])
df = df[(df['isfile1'] == True) & (df['isfile2'] == True)]
del df['fname1'],df['fname2'], df['isfile1'], df['isfile2']

# Calculate parameters for each larvae
def calculate_parameters(prefix, temp, df, n):
    ''' Add calculations to the master df given the data for larva #n in the file temp. '''
    
    df.loc[n, str(prefix + 'mean_speed_mm')] = np.mean(temp['speed_EL'].dropna())*2
    df.loc[n, str(prefix + '%_time_L_half')] = len(temp[temp['pos_x_mm'] < 40]) / len(temp)
    df.loc[n, str(prefix + '%_time_R_half')] = len(temp[temp['pos_x_mm'] > 40]) / len(temp)
    df.loc[n, str(prefix + 'PI_(L-R)')] = (df.loc[n, str(prefix + '%_time_L_half')] - \
                                                 df.loc[n, str(prefix + '%_time_R_half')]) / 1 # total time
    return(df)

df = df.reset_index()
for n in range(0, len(df)):
    try:
        # Acclimate video data calculations
        temp_fname = './gcamp_video_calculations/' + df.loc[n, 'animal_ID'] + '-acclimate.csv'
        temp = pd.read_csv(temp_fname)
        df = calculate_parameters('A_', temp, df, n)

        # Experiment video data calculations
        temp_fname = './gcamp_video_calculations/' + df.loc[n, 'animal_ID'] + '-experiment.csv'
        temp = pd.read_csv(temp_fname)
        df = calculate_parameters('E_', temp, df, n)
    except:
        print(temp_fname)

# Save to a new CSV file 
del df['index'] # temporary index for numbering df.loc calls. Animal_ID should be actual index. 
df.to_csv('./individual_larva_calculations_GCAMP.csv', index=False)
df.head()

Missing acclimation files
Series([], Name: animal_ID, dtype: object)
Missing experiment files
Series([], Name: animal_ID, dtype: object)


,animal_ID,experiment_date,acclimate_start,experiment_start,sex,dead,treatment_odor,starve_date,starve_time,background,A_mean_speed_mm,A_%_time_L_half,A_%_time_R_half,A_PI_(L-R),E_mean_speed_mm,E_%_time_L_half,E_%_time_R_half,E_PI_(L-R)
0,171018-01-top,171018,2:41:00 PM,2:56:00 PM,f,False,milliQ_water_100uL,171017,1:00:00 PM,orco5_gcamp,2.077219,0.628476,0.371524,0.256952,2.377931,0.763626,0.236374,0.527253
1,171018-01-bottom,171018,2:41:00 PM,2:56:00 PM,m,False,milliQ_water_100uL,171017,1:00:00 PM,orco5_gcamp,1.708343,0.506118,0.493882,0.012236,2.042520,0.195664,0.804336,-0.608671
2,171018-02-top,171018,3:17:00 PM,3:32:00 PM,f,False,milliQ_water_100uL,171017,1:00:00 PM,orco5_gcamp,1.259628,0.196329,0.803671,-0.607341,1.438006,0.741379,0.258621,0.482759
3,171018-02-bottom,171018,3:17:00 PM,3:32:00 PM,f,False,milliQ_water_100uL,171017,1:00:00 PM,orco5_gcamp,2.225063,0.642023,0.357977,0.284047,2.816649,0.315350,0.684650,-0.369299
4,171018-03-top,171018,3:52:00 PM,4:07:00 PM,NaN,True,milliQ_water_100uL,171017,1:00:00 PM,orco5_gcamp,1.753005,0.000000,1.000000,-1.000000,2.106719,0.473860,0.526140,-0.052280
